In [1]:
%run SequencingSimulator.ipynb

In [2]:
#test for getStandardDeviation function

assert getStandardDeviation(35) == 1
assert getStandardDeviation(125) == 1
assert getStandardDeviation(45) == 2
assert getStandardDeviation(115) == 2
assert getStandardDeviation(65) == 5
assert getStandardDeviation(95) == 5
assert getStandardDeviation(75) == 10

In [3]:
#test for generateQuality function

def calculateSum(readLength, quality, readQuality):
    sum = 0
    for ch in readQuality:
        sum += ord(ch)
    sum = (int)(sum/readLength)
    return sum
    
def generateQualityTest(quality, readLength):
    readQuality = generateQuality(quality, readLength)
    assert len(readQuality) == readLength
    sum = calculateSum(readLength, quality, readQuality)
    assert sum <= quality + 2
    assert sum >= quality - 2

readLength = 100
quality = 70
generateQualityTest(quality, readLength)

readLength = 150
quality = 50
generateQualityTest(quality, readLength)

In [4]:
#test for generateSingleRead function and getLeftMostPosition

refGenome = ['D','T','C','A','G','T','C','G',0,2]
readSize = 4
refPos = 2
direction = 1
readAndIndex = generateSingleRead(readSize, refGenome, refPos, direction, lambda x: x)
assert len(readAndIndex[0]) == 4
assert readAndIndex[0] == "CAGT"
assert readAndIndex[1] == 5
leftmostPosition = getLeftmostPosition(refGenome, refPos, readAndIndex[1])
assert leftmostPosition == 3

readSize = 4
refPos = 7
direction = -1
readAndIndex = generateSingleRead(readSize, refGenome, refPos, direction, lambda x: complNucleotids[x])
assert len(readAndIndex[0]) == 4
assert readAndIndex[0] == "CGAC"
assert readAndIndex[1] == 4
leftmostPosition = getLeftmostPosition(refGenome, refPos, readAndIndex[1])
assert leftmostPosition == 8

readSize = 5
refPos = 0
direction = 1
readAndIndex = generateSingleRead(readSize, refGenome, refPos, direction, lambda x: x)
assert len(readAndIndex[0]) == 5
assert readAndIndex[0] == "TCAGT"
assert readAndIndex[1] == 5
leftmostPosition = getLeftmostPosition(refGenome, refPos, readAndIndex[1])
assert leftmostPosition == 1

readSize = 4
refPos = 6
direction = 1
readAndIndex = generateSingleRead(readSize, refGenome, refPos, direction, lambda x: x)
assert len(readAndIndex[0]) == 4
assert readAndIndex[0] == "CGAG"
assert readAndIndex[1] == 9
leftmostPosition = getLeftmostPosition(refGenome, refPos, readAndIndex[1])
assert leftmostPosition == 7

refGenome = ['D',7,6,5,4,0]
readSize = 5
refPos = 0
direction = 1
readAndIndex = generateSingleRead(readSize, refGenome, refPos, direction, lambda x: x)
assert len(readAndIndex[0]) == 5
assert readAndIndex[0] == "TGCAA"
assert readAndIndex[1] == 5
leftmostPosition = getLeftmostPosition(refGenome, refPos, readAndIndex[1])
assert leftmostPosition == 1

In [5]:
#test for readGenome function and insertMutations function

fileName = "GenomTest.fa"
seq1length = 15
seq2length = 17
refGenomeDict = readGenome(fileName)
assert len(refGenomeDict) == 2
assert ("sequence1" in refGenomeDict) == True
assert len(refGenomeDict["sequence1"]) == seq1length
assert ("sequence2" in refGenomeDict) == True
assert len(refGenomeDict["sequence2"]) == seq2length

errorSnv = 0.1
errorInDel = 0.05
expectedNumOfMut = {}
for key, value in refGenomeDict.items(): 
    expectedNumOfMut[key] = round(len(value)*(errorSnv)) + round(len(value)*(errorInDel))

insertMutations(refGenomeDict, errorSnv, errorInDel)

for key, genome in refGenomeDict.items():
    actualNumOfMut = 0
    for nucleotid in genome:
        if isinstance(nucleotid, int) or nucleotid == 'D':
            actualNumOfMut += 1
    assert actualNumOfMut == expectedNumOfMut[key]

In [6]:
#test for validateParameters function

assert validateParameters(55, 5, 9, 10, 0.2, 0.1) == True
assert validateParameters(25, 5, 9, 10, 0.2, 0.1) == False
assert validateParameters(55, -5, 9, 10, 0.2, 0.1) == False
assert validateParameters(55, 5, -9, 10, 0.2, 0.1) == False
assert validateParameters(55, 5, 9, -10, 0.2, 0.1) == False
assert validateParameters(55, 5, 10, 9, 0.2, 0.1) == False
assert validateParameters(55, 5, 9, 10, -0.2, 0.1) == False
assert validateParameters(55, 5, 9, 10, 0.2, -0.1) == False
assert validateParameters(55, 5, 9, 10, 1.2, 0.1) == False
assert validateParameters(55, 5, 9, 10, 0.2, 1.1) == False
assert validateParameters(55, 5, 9, 10, 0.8, 0.7) == False

Quality must be whole number between 33 and 126. Please choose the correct value and try again
Coverage must be whole number bigger then 0. Please choose the correct value and try again.
ReadSize must be whole number bigger then 0. Please choose the correct value and try again.
InsertSize must be whole number bigger then 0. Please choose the correct value and try again.
ReadSize cannot be longer than InsertSize. Please choose the correct value and try again.
ReadSize cannot be longer than InsertSize. Please choose the correct value and try again.
ErrorSNV must represent probability between 0 and 1. Please choose the correct value and try again.
ErrorINDEL must represent probability between 0 and 1. Please choose the correct value and try again.
ErrorSNV must represent probability between 0 and 1. Please choose the correct value and try again.
Sum of error rates for SNV  and INDEL cannot be larger than 1. Please choose the correct values and try again.
ErrorINDEL must represent probabil

In [7]:
#test for simulatePairedEndSequencing function

readLength = 7
simulatePairedEndSequencing("GenomTest.fa", 70, 3, readLength, 12)
samFile = open("GenomTest.sam")
i = 0
for line in samFile.readlines():
    if len(line) != 0:
        if i == 0:
            assert line == "@SQ SN:sequence1 LN:15\n"
            i += 1
        elif i == 1:
            assert line == "@SQ SN:sequence2 LN:17\n"
            i += 1
        else:
            words = line.split()
            assert len(words) == 4
            assert len(words[2]) == readLength
            assert len(words[3]) == readLength
            i += 1
assert i == (2 + 6 + 8) #2 - there are 2 sequences, 6  - there are 6 reads from first sequence, 8- there are 8 reads from second sequence
samFile.close()